In [1]:
# Packages to be used
import networkx as nx
from networkx.algorithms import approximation
from networkx.algorithms import community
from networkx.algorithms import centrality
import numpy as np
import matplotlib.pyplot as plt
import ndlib.models.epidemics as ep
import ndlib.models.ModelConfig as mc
import random
# import scipy
# import math
# import sys
import pandas as pd
# from fractions import Fraction
# import timeit
# from time import sleep
import igraph as ig
import time
import collections
# import heapq as heap
# from tqdm import tqdm 
import community
from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend

no display found. Using non-interactive Agg backend


In [2]:
# Load datasets, outside of definitions

# G1 = nx.read_edgelist("Wiki-Vote.txt", create_using=nx.DiGraph(), nodetype= int)
# print(G1)
G2 = nx.read_edgelist("Wiki-Vote.txt", create_using=nx.Graph(), nodetype=int)
print(G2)


Graph with 7115 nodes and 100762 edges


In [ ]:
# Making a directed graph undirected

G1_undirected = G1.to_undirected()
print(G1_undirected)

Heuristics to block/remove nodes

In [3]:
# Centrality measures

# Degree
sort_degree = sorted(dict(G2.degree).items(), key=lambda item: item[1], reverse=True)
# print(sort_degree)

# Degree centrality
degree_centrality = nx.centrality.degree_centrality(G2)
sort_degree_centrality = sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True)
# print(sort_degree_centrality)

# Betweenness

betweenness = nx.betweenness_centrality(G2)
sort_betweenness = sorted(betweenness.items(), key=lambda item: item[1], reverse=True)
# print(sort_betweenness)

# Closeness

closeness = nx.closeness_centrality(G2)
sort_closeness = sorted(closeness.items(), key=lambda item: item[1], reverse=True)
# print(sort_closeness)


# Influence maximization
# Pagerank

pagerank = nx.pagerank(G2)
sort_pagerank = sorted(pagerank.items(), key=lambda item: item[1], reverse=True)
# print(sort_pagerank)

# Eigenvector

eigenvector = nx.eigenvector_centrality(G2)
sort_eigenvector = sorted(eigenvector.items(), key=lambda item: item[1], reverse=True)
# print(sort_eigenvector)

In [71]:
# To only get the keys from the sorted list

only_keys = [k for k,v in sort_degree]
print(only_keys)

# Take the top 50 nodes to be used as seeds later
first_50 = only_keys[0:50]
print(first_50)

[2565, 766, 11, 1549, 457, 1166, 2688, 1374, 1151, 5524, 3352, 4037, 737, 2485, 2328, 3456, 2871, 5802, 1608, 15, 2651, 5079, 1133, 2972, 2237, 2398, 789, 4310, 3453, 2658, 3642, 5189, 4967, 1305, 993, 1297, 311, 173, 2256, 2625, 3447, 5022, 5800, 6, 2576, 3449, 5697, 722, 762, 24, 1098, 72, 813, 4712, 4828, 5179, 996, 988, 5254, 2654, 2326, 4099, 2790, 1542, 600, 3976, 68, 407, 4191, 4632, 3787, 310, 826, 2660, 3459, 2535, 2145, 5543, 306, 1496, 2411, 5188, 3026, 4045, 3028, 1428, 2066, 36, 4335, 3568, 2297, 1211, 3614, 3089, 825, 4536, 784, 2713, 5531, 2653, 6907, 2967, 5020, 1319, 2369, 6305, 2560, 28, 3586, 2693, 1492, 3334, 5814, 2323, 312, 5002, 3796, 1919, 7092, 1972, 3615, 2859, 3454, 4441, 5412, 86, 1210, 8, 1055, 2700, 3032, 2135, 4948, 122, 319, 1167, 4483, 3641, 3394, 6327, 7047, 3381, 7052, 7620, 2474, 6914, 2686, 2516, 2120, 4653, 1717, 6634, 4899, 1922, 3276, 432, 2240, 3084, 6596, 1395, 2657, 2667, 1633, 4735, 87, 290, 1772, 663, 3897, 1186, 3537, 2371, 271, 1808, 922, 

In [ ]:
# Create a copy of the original graph to be modified later, by removing nodes
G1_copy = G1.copy()
print(G1_copy)

G1_copy.remove_nodes_from(first_10)

In [ ]:
# Miscellaneous things and different heuristics

# maximum_clique = nx.approximation.max_clique(g)
centrality = nx.centrality.degree_centrality(G1)
# value = list(centrality.values())[0]
# avg_cluster_G2 = nx.approximation.average_clustering(G2, trials=1000, seed=10)
# print(avg_cluster_G2)
# G1.nodes[30]["centrality"]

In [ ]:
# Adding weight to the edges of the graph

for u,v in G1.edges():
    G1[u][v]['weight'] = 1/G1.in_degree(v)
G1.get_edge_data(30,1412)
G1.in_degree(1412)

# for u,v in G2.edges():
#     G2[u][v]['weight'] = 1/G2.degree(v)
# G2.get_edge_data(0,5)


In [ ]:
# Adds an attribute to the nodes of the graph, for example degree centrality

nx.set_node_attributes(G1, centrality, "centrality")
# nx.set_node_attributes(G2, centrality, "centrality")

In [ ]:
# Filtering the graph by removing edges based on their weight attribute

for u,v in list(G1.edges()):
    if G1[u][v]['weight'] < 0.04:
        G1.remove_edge(u,v)
G1_new = G1.copy()

In [ ]:
# Remove nodes based on their degree

for u in list(G1.nodes()):
    if G1.degree(u)< 10:
        G1.remove_node(u)

In [ ]:
# Remove nodes based on their degree centrality

for u,v in centrality.items():
    if v < 0.004:
        G1.remove_node(u)

In [ ]:
# Lists of sources and targets based on in/out degrees of nodes

sources = []
targets = []
for u in G1.nodes():
    if G1.in_degree(u) == 0:
        sources.append(u)
for v in G1.nodes():
    if G1.out_degree(v) == 0:
        targets.append(v)

In [ ]:
# Splitting the sources to smaller components (useful for seed selection)

sub_sources = sources[0:50]
print(sub_sources)


In [ ]:
G1_weighted = nx.read_edgelist("Wiki-Vote-edge_weight.edgelist", create_using=nx.DiGraph(), nodetype= int)

In [ ]:
ranking1 = nx.pagerank(G1_new, weight = "weight")
pagerank = nx.pagerank(G1, weight = 'weight')
print(ranking1)
# max_value = min(pagerank.values())
# print(max_value)

max_value = max(ranking1.values())
print(max_value)
print(pagerank[94], pagerank[1412], pagerank[4037], pagerank[25])

In [ ]:
# Working with the .glm version of the Wiki-Vote

# G1_graphml.get_edge_data("30","1412")
# G1_graphml.nodes["30"]["centrality"]

In [ ]:
# G = Graph.Erdos_Renyi(n=100,m=300,directed=True)
Gnew = Graph.Load("facebook-edge_node_attr.gml", format='gml')


In [ ]:
# Read gml graph
G_new = ig.Graph.Read_GML("facebook-edge_node_attr.gml")

In [14]:
import random
def random(nodes, n):
    ''' Select seeds randomly
    Args:
        nodes (list) [#node]: node list of the graph;
        n (int): number of seeds;
    Returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    # random.sample(nodes, n)
    # return nodes[:n]
    # np.random.shuffle(nodes)
    # return nodes[:n]
    seeds = random.sample(nodes, n)
    return seeds

In [8]:
def mia(nodes, edges, n):
    ''' select seeds by mia policy
    args:
        nodes (list) [#node]: node list of the graph;
        edges (list of list) [#edge, 2]: edge list of the graph;
        n (int): number of seeds;
    returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    out_connection = {}
    in_connection = {}
    centrality_score = {}
    seeds = []
    for edge in edges:
        if edge[0] in out_connection:
            out_connection[edge[0]].append(edge[1])
        else:
            out_connection[edge[0]] = [edge[1]]
        if edge[1] in in_connection:
            in_connection[edge[1]] += 1
        else:
            in_connection[edge[1]] = 1
    for node in nodes:
        centrality_score[node] = mia_centrality(node, out_connection, in_connection)
    i = 0
    for node, _ in sorted(centrality_score.items(), key=lambda item: item[1], reverse=True):
        if i >= n:
            break
        else:
            i += 1
        seeds.append(node)
    return seeds

def mia_centrality(node, out_connection, in_connection):
    ''' select seeds by mia centrality policy
    '''
    theta = 0.5
    c_score = 0
    q = 1  # threshold of influence
    visited = set()
    path_prob = 1
    c_score = dfs(visited, out_connection, path_prob, in_connection, node, theta, q)
    return c_score

def dfs(visited, out_connection, path_prob, in_connection, node, theta, q):
    if node not in visited:
        visited.add(node)
        if node in out_connection:
            for neighbour in out_connection[node]:
                path_prob *= q / in_connection[neighbour]
                if path_prob >= theta:
                    dfs(visited, out_connection, path_prob, in_connection, neighbour, theta, q)
                    path_prob /= (q / in_connection[neighbour])
                else:
                    path_prob /= (q / in_connection[neighbour])
    N_of_nodes = len(visited)
    return N_of_nodes

In [3]:
def degree(edges, n):
    ''' Select seeds by degree policy
    Args:
        edges (list of list) [#edge, 2]: edge list of the graph;
        n (int): number of seeds;
    Returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    degree = {}
    seeds = []
    for edge in edges:
        if edge[0] in degree:
            degree[edge[0]] += 1
        else:
            degree[edge[0]] = 1
        if edge[1] in degree:
            degree[edge[1]] += 1
        else:
            degree[edge[1]] = 1
    seeds = list({k: v for k, v in sorted(degree.items(), key=lambda item: item[1], reverse=True)}.keys())[:n]
    return seeds

In [7]:
def degree_discount(edges, n):
    ''' Select seeds by degree discount degree
    Args:
        edges (list of list) [#edge, 2]: edge list of the graph;
        n (int): number of seeds;
    Returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    out_degree = {}
    connection = {}
    seeds = []
    for edge in edges:
        if edge[1] in connection:
            connection[edge[1]].append(edge[0])
        else:
            connection[edge[1]] = [edge[0]]
        if edge[0] in out_degree:
            out_degree[edge[0]] += 1
        else:
            out_degree[edge[0]] = 1
    while len(seeds) < n:
        seed = sorted(out_degree.items(), key=lambda item: item[1], reverse=True)[0][0]
        seeds.append(seed)
        out_degree[seed] = -1
        if seed in connection:
            for node in connection[seed]:
                if node in out_degree:
                    out_degree[node] -= 1
    return seeds

In [19]:
def degree_neighbor_fix(edges, n):
    ''' select seeds by degree neighbor fix policy
    args:
        edges (list of list) [#edge, 2]: edge list of the graph;
        n (int): number of seeds;
    returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    out_degree = {}
    centrality_score = {}
    connection = {}
    seeds = []
    for edge in edges:
        if edge[1] in connection:
            connection[edge[1]].append(edge[0])
        else:
            connection[edge[1]] = [edge[0]]
        if edge[0] in out_degree:
            out_degree[edge[0]] += 1
        else:
            out_degree[edge[0]] = 1
    centrality_score = out_degree.copy()
    for edge in edges:
        if edge[1] in out_degree:
            centrality_score[edge[0]] += out_degree[edge[1]]
    while len(seeds) < n:
        seed = sorted(centrality_score.items(), key=lambda item: item[1], reverse=True)[0][0]
        seeds.append(seed)
        centrality_score[seed] = -1
        if seed in connection:
            for node in connection[seed]:
                if node in out_degree:
                    centrality_score[node] -= out_degree[seed]

    return seeds

In [14]:
def degree_neighbor(edges, n):
    ''' select seeds by degree neighbor policy
    args:
        edges (list of list) [#edge, 2]: edge list of the graph;
        n (int): number of seeds;
    returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    out_degree = {}
    centrality_score = {}
    seeds = []
    for edge in edges:
        if edge[0] in out_degree:
            out_degree[edge[0]] += 1
        else:
            out_degree[edge[0]] = 1
    centrality_score = out_degree.copy()
    for edge in edges:
        if edge[1] in out_degree:
            centrality_score[edge[0]] += out_degree[edge[1]]
    seeds = sorted(centrality_score.items(), key=lambda item: item[1], reverse=True)
    return seeds


In [13]:
# Create communities based on the louvain method

louvain = nx.algorithms.community.louvain_communities(G2)
print(len(louvain))
# print(louvain[131])

35


In [14]:
# Create a seed set based on the communities and a heuristic

seeds_new = {}
for community in louvain:
    max_node = max(community, key=lambda node: degree_centrality[node])
    seeds_new[max_node] = community

In [ ]:
# policy = 'degree'

# seeds_number = int(len(G1.nodes) * init_rate)
# seeds = list({k: v for k, v in sorted(dict(degreenew).items(), key=lambda item: item[1], reverse=True)}.keys())[:seeds_number]
policy = 'mia'

init_rate = 0.02
# nodes = G2_new.nodes
# edges = G2_new.edges
# # nodes = G_new.vs
# # edges = G_new.es
nodes = G2.nodes()
edges = G2.edges()
seeds_number = int(len(nodes) * init_rate)

if policy == 'degree':
    seeds = degree(edges, seeds_number)
    # seeds = list({k: v for k, v in sorted(dict(new_degree).items(), key=lambda item: item[1], reverse=True)}.keys())[:seeds_number]
if policy == 'random':
    seeds = random(nodes, seeds_number)
if policy == 'degree_discount':
    seeds = degree_discount(edges, seeds_number)
if policy == 'mia':
   seeds = mia(nodes, edges, seeds_number) 
if policy == 'degree_neighbor_fix':
    seeds = degree_neighbor_fix(edges, seeds_number)
if policy == 'degree_neighbor':
    seeds = degree_neighbor(edges, seeds_number)
print(f'Number of seeds: {len(seeds)}')

In [ ]:
print(seeds)

Use louvain communities without fairness measure and different centrality measures

In [ ]:
import community

# Partition into communities
partition = community.best_partition(G2)
for node in G2.nodes:
    G2.nodes[node]['community'] = partition[node]
print(len(partition))

num_distinct_values = len(set(partition.values()))

print("The dictionary has", num_distinct_values, "distinct values.")

In [34]:


# Calculate centralities (can also be done from the cell above, have to check to decide)

seeds = set()
for community_id in set(partition.values()):
    community_nodes = [node for node in G2.nodes if G2.nodes[node]['community'] == community_id]
    max_degree_node = max(community_nodes, key=lambda node: pagerank[node])
    seeds.add(max_degree_node)
print("Number of seeds:", len(seeds))


Number of seeds: 30


Use louvain communities with equality fairness and different centrality measures

In [3]:
import community

# Partition into communities
partition = community.best_partition(G2)
print(len(partition))
betweenness_centralities = {}
closeness_centralities = {}
degree_centralities = {}
eigenvector_centralities = {}
pagerank_centralities = {}
for community_id in set(partition.values()):
    community_nodes = [n for n in partition.keys() if partition[n] == community_id]
    community_subgraph = G2.subgraph(community_nodes)
    # betweenness_centralities[community_id] = nx.betweenness_centrality(community_subgraph)
    # degree_centralities[community_id] = nx.degree_centrality(community_subgraph)
    # closeness_centralities[community_id] = nx.closeness_centrality(community_subgraph)
    # eigenvector_centralities[community_id] = nx.eigenvector_centrality(community_subgraph)
    pagerank_centralities[community_id] = nx.pagerank(community_subgraph)

# Calculate fairness metric
fairness_metrics = {}
for community_id in set(partition.values()):
    # fairness = max(betweenness_centralities[community_id].values()) - min(betweenness_centralities[community_id].values())
    # fairness = max(degree_centralities[community_id].values()) - min(degree_centralities[community_id].values())
    # fairness = max(closeness_centralities[community_id].values()) - min(closeness_centralities[community_id].values())       
    # fairness = max(eigenvector_centralities[community_id].values()) - min(eigenvector_centralities[community_id].values())       
    fairness = max(pagerank_centralities[community_id].values()) - min(pagerank_centralities[community_id].values())                                                          
                                                   
    fairness_metrics[community_id] = fairness
    
# Calculate seeds from each community
seeds = []
min_fairness_community = min(fairness_metrics, key=fairness_metrics.get)

for community_id in set(partition.values()):
    community_nodes = [n for n in partition.keys() if partition[n] == community_id]
    community_subgraph = G2.subgraph(community_nodes)

    if community_id == min_fairness_community:
        # node = max(betweenness_centralities[community_id], key=betweenness_centralities[community_id].get)
        # node = max(degree_centralities[community_id], key=degree_centralities[community_id].get)
        # node = max(closeness_centralities[community_id], key=closeness_centralities[community_id].get)
        # node = max(eigenvector_centralities[community_id], key=eigenvector_centralities[community_id].get)
        node = max(pagerank_centralities[community_id], key=pagerank_centralities[community_id].get)

    else:
        # nodes_sorted = sorted(betweenness_centralities[community_id].items(), key=lambda x: x[1], reverse=True)
        # nodes_sorted = sorted(degree_centralities[community_id].items(), key=lambda x: x[1], reverse=True)
        # nodes_sorted = sorted(closeness_centralities[community_id].items(), key=lambda x: x[1], reverse=True)
        # nodes_sorted = sorted(eigenvector_centralities[community_id].items(), key=lambda x: x[1], reverse=True)
        nodes_sorted = sorted(pagerank_centralities[community_id].items(), key=lambda x: x[1], reverse=True)
        node = nodes_sorted[0][0]

    seeds.append(node)
print("Number of seeds:", len(seeds))

7115
Number of seeds: 30


The "correct" implementation for the equality fairness is bellow (as implemented in Farnandi's paper)

In [8]:
import community
# Calculcate communities using louvain method
partition = community.best_partition(G2)

# Create a dictionary to store the population ratio of each community:

community_sizes = {}
for c in set(partition.values()):
    nodes_in_community = [n for n in partition.keys() if partition[n] == c]
    community_sizes[c] = len(nodes_in_community)

# Calculate the centralities for each node in the graph

# closeness_centrality = nx.closeness_centrality(G2)
# betweenness_centrality = nx.betweenness_centrality(G2)
# degree_centrality = nx.degree_centrality(G2)
# eigen_centrality = nx.eigenvector_centrality(G2)
pagerank_centrality = nx.pagerank(G2)

# Create a dictionary to store the centralities for each community:

# community_closeness = {}
# community_betweenness = {}
# community_degree = {}
# community_eigen = {}
community_pagerank = {}
for node in G2.nodes():
    community = partition[node]
    # if community not in community_closeness:
    #     community_closeness[community] = 0
    # if community not in community_betweenness:
    #     community_betweenness[community] = 0
    # if community not in community_degree:
    #     community_degree[community] = 0
    # if community not in community_eigen:
    #     community_eigen[community] = 0
    if community not in community_pagerank:
        community_pagerank[community] = 0

    # community_closeness[community] += closeness_centrality[node]
    # community_betweenness[community] += betweenness_centrality[node]
    # community_degree[community] += degree_centrality[node]
    # community_eigen[community] += eigen_centrality[node]
    community_pagerank[community] += pagerank_centrality[node]


# Calculate the total centrality for each community

# for community in community_closeness:
#     community_closeness[community] /= community_sizes[community]
# for community in community_betweenness:
#     community_betweenness[community] /= community_sizes[community]
# for community in community_degree:
#     community_degree[community] /= community_sizes[community]
# for community in community_eigen:
#     community_eigen[community] /= community_sizes[community]
for community in community_pagerank:
    community_pagerank[community] /= community_sizes[community]

# Calculate the number of seeds for each community proportional to the population ratio using the centralities:
seeds = {}
for node in G2.nodes():
    community = partition[node]
    if community not in seeds:
        seeds[community] = []
        # if len(seeds[community]) < community_sizes[community] * community_closeness[community]:
            # seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_betweenness[community]:
        #     seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_degree[community]:
        #     seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_eigen[community]:
        #     seeds[community].append(node)
        if len(seeds[community]) < community_sizes[community] * community_pagerank[community]:
            seeds[community].append(node)

print("Number of seeds:", len(seeds))

Number of seeds: 31


In [16]:
seeds_random = [766, 2565, 457, 11, 1549, 2688, 1166, 1151, 1374, 311, 2485, 813, 1133, 3453, 1608, 24, 2967, 988, 1098, 2972, 789, 4967, 2237, 3447, 5524, 3352, 173, 3456, 68, 5079, 993, 2651, 36, 3641, 306, 3449, 3642, 1305, 2328, 784, 2871, 722, 2658, 3032, 5802, 3787, 122, 1210, 1542, 4045, 2326, 4310, 996, 5179, 5531, 1496, 5189, 600, 310, 825, 1319, 3026, 5697, 879, 1972, 2713, 6, 3614, 87, 5796, 826, 3394, 3028, 2135, 2686, 319, 5188, 1470, 2297, 4099, 312, 109, 663, 1055, 2256, 4632, 20, 2653, 72, 634, 3454, 5800, 1492, 1062, 1922, 2145, 1395, 737, 2660, 1167, 2674, 2842, 2790, 5543, 769, 1428, 2693, 3452, 1808, 4948, 3666, 2700, 3457, 432, 2560, 5002, 3615, 317, 2323, 2369, 3010, 922, 2229, 3976, 4483, 2371, 1521, 546, 3586, 4441, 1137, 172, 3568, 6305, 1583, 2940, 695, 308, 4065, 2474, 1522, 707]

print(len(seeds_random))

142


In [7]:
# This code can be used with the previous two cell blocks to calculate the influence count. 
# It combines the method of calculating the spread using spreading model as defined in the NDlib package, witht the different heuristic approaches.
average = []
for index in range(6):
    # model = ep.IndependentCascadesModel(G2)
    # model = ep.SIRModel(G2)
    # model = ep.SIModel(G2)
    model = ep.ThresholdModel(G2)
    cfg = mc.Configuration()
    # cfg.add_model_parameter('beta', 0.01)
    # cfg.add_model_parameter('gamma', 0.005)
    # cfg.add_model_parameter('lambda', 0.005)
    infected_nodes = seeds
    cfg.add_model_initial_configuration("Infected", infected_nodes)
    # cfg.add_model_parameter("fraction_infected", 0.05)
    model.set_initial_status(cfg)
    # threshold = 0.2
    # for e in G2.edges():
    #     cfg.add_edge_configuration("threshold", e, random.uniform(0,1))
    for n in G2.nodes():
        cfg.add_node_configuration("threshold", n, 0.01)

    # iterations = model.iteration_bunch(500, True, True)
    # trends = model.build_trends(iterations)
    # viz = DiffusionTrend(model, trends)
    # p = viz.plot(width=800, height=800)
    # show(p)
    list = []
    for i in range(0,1000):
        iterations = model.iteration_bunch(1)

        dict = iterations[0]
        for i in dict:
            k = (i,dict[i])
            list.append(k)
    list2 = []
    for i in range(len(list)):
        if list[i][0] == 'status':
            list2.append(list[i][1])
    list4 = []
    for t in range(len(list2)):
        for f in list2[t]:
            if list2[t][f] == 1:
                list4.append(f)
    # print("================================")
    # print(f"Results for iteration {index+1}:")
    # print("Total nodes infected: ", len(list4), "out of", len(G2.nodes()))
    percentage = (len(list4)/len(G2.nodes()))*100
    # print("Percentage of total infected nodes with current seed set: ", percentage, "%")
    # print("================================")
    average.append(percentage)
print("The average percentage is: ", sum(average)/len(average), "%")

The average percentage is:  26.188802998360273 %


In [14]:
print(average)

[72.46661981728742, 76.58468025298664, 61.48981026001405, 72.53689388615601]
